In [16]:
import pandas as pd
import zipfile
import os

import tempfile
import shutil

# Caminho onde estão os arquivos .zip
caminho = 'dados/GTFS'

# Lista dos arquivos GTFS
arquivos_zip = [f for f in os.listdir(caminho) if f.endswith('.zip')]


In [17]:
# DataFrames acumuladores
df_trips = []
df_routes = []
df_stop_times = []
df_stops = []

# Iterar sobre cada arquivo .zip
for arquivo in arquivos_zip:
    data_ref = arquivo.replace('google_transit_', '').replace('.zip', '')
    caminho_arquivo = os.path.join(caminho, arquivo)
    
    with zipfile.ZipFile(caminho_arquivo, 'r') as z:
        if 'trips.txt' in z.namelist():
            df = pd.read_csv(z.open('trips.txt'))
            df['data_referencia'] = data_ref
            df_trips.append(df)
        
        if 'routes.txt' in z.namelist():
            df = pd.read_csv(z.open('routes.txt'))
            df['data_referencia'] = data_ref
            df_routes.append(df)
        
        if 'stop_times.txt' in z.namelist():
            df = pd.read_csv(z.open('stop_times.txt'))
            df['data_referencia'] = data_ref
            df_stop_times.append(df)
        
        if 'stops.txt' in z.namelist():
            df = pd.read_csv(z.open('stops.txt'))
            df['data_referencia'] = data_ref
            df_stops.append(df)

# Concatenar os dados
trips_df = pd.concat(df_trips, ignore_index=True)
routes_df = pd.concat(df_routes, ignore_index=True)
stop_times_df = pd.concat(df_stop_times, ignore_index=True)
stops_df = pd.concat(df_stops, ignore_index=True)

stop_times_df=stop_times_df[['trip_id','stop_id','stop_sequence','data_referencia']].drop_duplicates()
trips_df['route_id']=trips_df['route_id'].str.replace("-","")
routes_df['route_id']=routes_df['route_id'].str.replace("-","")
services={'USD':"Todos", 'U__':"Úteis", 'US_':"Úteis e Sábados", '_SD':"Sábados e Domingos", '__D':"Domingos", '_S_':"Sábados"}
trips_df['service_id']=trips_df['service_id'].map(services)

In [18]:
trips_df=trips_df.merge(stop_times_df,on=['trip_id','data_referencia']).merge(stops_df,on=['stop_id','data_referencia'])


In [20]:
routes_df

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color,data_referencia
0,101510,1,1015-10,Chácara Maria Trindade - Terminal Jd. Britania,3,NaN,NaN,201501
1,101610,1,1016-10,Cemiterio Do Horto - Center Norte,3,NaN,NaN,201501
2,101710,1,1017-10,Perus - Vila Iorio,3,NaN,NaN,201501
3,101810,1,1018-10,Vila Rosa - Metrô Santana,3,NaN,NaN,201501
4,102110,1,1021-10,Cohab Brasilândia - Terminal Pirituba,3,NaN,NaN,201501
...,...,...,...,...,...,...,...,...
167341,N83911,1,N839-11,Metrô Butantã - Metrô Vl. Mariana,3,FF671F,000000,202503
167342,N84011,1,N840-11,Term. Vl. Mariana - Sta. Cecília,3,FF671F,000000,202503
167343,N84111,1,N841-11,Term. Vl. Mariana - Sta. Cecília,3,FF671F,000000,202503
167344,N84211,1,N842-11,Term. Pinheiros - Cohab Raposo Tavares,3,FF671F,000000,202503


In [19]:
routes_df.to_parquet('dados/rotaaas_historico.parquet', index=False)
trips_df.to_parquet('dados/viagens_historico.parquet', index=False)
